In [1]:
#Imports the libraries and read the data files
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os, sys, email
import gensim
from gensim.models import Doc2Vec
from sklearn.cluster import AffinityPropagation
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pprint
import nltk
from string import punctuation
import timeit
from sklearn.cluster import KMeans
from sklearn import metrics
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
start = timeit.default_timer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kaushlesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
repo_df = pd.read_csv('./news_tagged.csv')

In [3]:
repo_df.head()

,News_Heading,Risk Tag
0,IMF chief Lagarde charged over French corrupt...,F
1,$1 Billion Agreement with SunTrust to Address ...,L
2,$1 million for GM victims' families,L
3,$1 million IRS phone scam 'largest ever',F
4,$1.2 B settlement with Toyota because of safet...,L


In [4]:
repo_df = repo_df.drop(['Unnamed: 1','Unnamed: 2'], axis = 1)

KeyError: "['Unnamed: 1' 'Unnamed: 2'] not found in axis"

In [5]:
sentences_df = pd.read_csv('./news_tagged.csv')

In [6]:
sentences_df.head()
print(sentences_df[:10])

                                        News_Heading Risk Tag
0   IMF chief Lagarde charged over French corrupt...        F
1  $1 Billion Agreement with SunTrust to Address ...        L
2                $1 million for GM victims' families        L
3           $1 million IRS phone scam 'largest ever'        F
4  $1.2 B settlement with Toyota because of safet...        L
5  $1.2 Billion Settlement For Toyota's 2009-2010...        L
6  $1.2 billion Toyota penalty sends warning to a...        L
7  $1.27 Billion in Damages to be Paid by the Ban...        L
8  $1.2B fine to settle probe delivers relief for...        L
9                    $1.2B Toyota fine: GM's future?        L


In [7]:
sentences_df['Risk Tag'].value_counts()

O    5154
I    2190
E    1376
F    1249
K     793
G     528
L     489
N     480
P     450
S      84
C      76
T      54
H      37
A      25
M      17
Q       9
Name: Risk Tag, dtype: int64

In [8]:
#sentences_df = sentences_df.drop(['Unnamed: 1','Unnamed: 2'], axis = 1)

In [9]:
sentences_df.iloc[0]

News_Heading     IMF chief Lagarde charged over French corrupt...
Risk Tag                                                        F
Name: 0, dtype: object

In [10]:
#Data cleaning
def sentence_cleaning(text):
    sentence = text.lower()    
    # clean and tokenize document string
    sentence_content = sentence.split()    
    word_list = []
    for i in sentence_content:
        x = 0
        if (('http' not in i) and ('@' not in i) and ('<.*?>' not in i) and i.isalnum() and (not i in stop_words)):
            word_list += [i]
        
    return word_list

#Data Pre-processing
def preprocessing(text):    
    # remove numbers
    number_tokens = [re.sub(r'[\d]', ' ', i) for i in text]
    number_tokens = ' '.join(number_tokens).split()
     # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in number_tokens]
    # remove empty
    length_tokens = [i for i in stemmed_tokens if len(i) > 1]
    return length_tokens

In [11]:
LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
all_content = []
#texts = []
j=0
k=0
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
p_stemmer = PorterStemmer()
for em in sentences_df.News_Heading:           
    #Data cleaning
    clean_content = sentence_cleaning(em)
    
    #Pre-processing
    processed_sentences = preprocessing(clean_content)
    #processed_sentences = clean_content
    
    # add tokens to list
    if processed_sentences:
        all_content.append(LabeledSentence1(processed_sentences,[k]))
        j+=1
        
    k+=1

print("Number of sentences processed: ", k)
print("Number of non-empty sentence vectors: ", j)

Number of sentences processed:  13011
Number of non-empty sentence vectors:  13006


In [12]:
print(all_content[0])
type(all_content)

TaggedDocument(['imf', 'chief', 'lagard', 'charg', 'french', 'corrupt', 'case'], [0])


list

In [13]:
all_content_repo = []
#texts_repo = []
j1=0
k1=0
for em1 in repo_df.News_heading:           
    #Data cleaning
    clean_content = sentence_cleaning(em1)
    
    #Pre-processing
    processed_sentences = preprocessing(clean_content)
    #processed_sentences = clean_content
    
    # add tokens to list
    if processed_sentences:
        all_content_repo.append(LabeledSentence1(processed_sentences,[k1]))
        j1+=1
        
    k1+=1

print("Number of sentences processed: ", k1)
print("Number of non-empty sentence vectors: ", j1)

AttributeError: 'DataFrame' object has no attribute 'News_heading'

In [16]:
d2v_model = Doc2Vec(vector_size = 50, window = 2, min_count = 1, workers=4, dm = 0, dbow_words = 1, negative = 15, alpha=0.025, min_alpha=0.0125)

In [17]:
d2v_model.build_vocab(all_content)

In [18]:
    from random import shuffle
    def randomize(sentence):
        x = [sentence[i] for i in range(len(sentence))]
        shuffle(x)
        return x
    

In [19]:
for epoch in range(10):
    d2v_model.train(randomize(all_content), total_examples=d2v_model.corpus_count, epochs=epoch, start_alpha=0.002, end_alpha=-0.016)

In [20]:
d2v_model.most_similar('sold')

/home/kaushlesh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('anoth', 0.9999213814735413),
 ('south', 0.9999160766601562),
 ('account', 0.9999148845672607),
 ('tweet', 0.9999130368232727),
 ('uber', 0.9999119639396667),
 ('crash', 0.9999107122421265),
 ('agre', 0.9999098181724548),
 ('strike', 0.9999096989631653),
 ('stapl', 0.999908447265625),
 ('recal', 0.9999083876609802)]

In [21]:
tokens = preprocessing(sentence_cleaning('Chinese Premier zero tolerance for corruption'))
print(tokens)
new_vector = d2v_model.infer_vector(tokens)
print(new_vector)
similar_doc = d2v_model.docvecs.most_similar([new_vector])
similar_doc = np.array(similar_doc)
print(similar_doc[:,0])
for l in similar_doc[:,0]:
    print(all_content[int(l)])

['chines', 'premier', 'zero', 'toler', 'corrupt']
[ 0.2624578   1.3895468   0.16779482  0.19994067 -0.09725436  0.94378054
  0.75570565  0.3058287  -1.33685     0.92304075  0.3186709  -0.4069424
  0.280578    0.5252728  -1.0196447   0.36567414  1.0893841  -0.15453741
  0.5716788   0.04749244 -0.72223175  0.70950115  0.05730813  0.3556812
 -0.90204227 -0.08934369  0.3693788  -0.90478927 -0.54129165 -0.42350915
  0.09821951 -0.5896394   0.31342548 -1.0418097   0.9336631  -0.0171691
  0.4707267   0.02140979  1.3152914   0.8247044  -0.09466263 -1.0133979
  0.04217716 -0.25827822 -0.5570872  -0.8104991  -0.5043073  -0.43172947
  0.3076577   0.16088198]
[ 6571.   558. 10849. 12188.   617.  5534. 11075.  3493.  1255.  6063.]
TaggedDocument(['nest', 'recal', 'smoke', 'alarm', 'safeti', 'risk'], [6574])
TaggedDocument(['view', 'crowdfund', 'environment'], [559])
TaggedDocument(['say', 'halifax', 'protest'], [10853])
TaggedDocument(['ico', 'probe', 'facebook', 'controversi', 'user', 'experi'], [

In [22]:
#saving the created model
d2v_model.save('d2v_model')
print('model saved')

model saved


In [23]:
import gc
gc.collect()

40

In [24]:
d2v_model.delete_temporary_training_data()

In [25]:
model_loaded = Doc2Vec.load('d2v_model')

In [26]:
clustering = AffinityPropagation()

In [27]:
clustering

AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)

In [28]:
clustering.fit(model_loaded.docvecs.vectors_docs)

AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)

In [29]:
labels=clustering.labels_.tolist()
type(labels)

list

In [30]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(d2v_model.docvecs.doctag_syn0, labels, metric='sqeuclidean'))

/home/kaushlesh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  


Silhouette Coefficient: 0.014


In [31]:
from sklearn import svm

In [38]:
clustering.

<bound method BaseEstimator.get_params of AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)>

In [26]:
len(sentences_df)

13011

In [27]:
len(labels)

13011

In [28]:
lab = pd.Series(labels)
#sentences_df['cluster']=lab.values
#sen = { 'Tagged Sentences': sentences_df, 'cluster': labels}

#frame = pd.DataFrame(sen)
sentences_clustered = sentences_df
sentences_clustered['cluster'] = lab.values

sentences_clustered['cluster'].value_counts()

104    210
5      203
31     195
48     181
24     174
60     172
44     171
81     169
27     163
94     163
29     161
80     159
65     158
50     158
9      158
111    156
1      154
58     153
55     153
8      152
95     147
72     146
23     146
18     144
16     144
92     144
83     143
42     142
98     142
28     142
      ... 
10     102
33     102
40     101
109     99
69      98
77      97
62      96
73      95
103     94
13      93
75      93
6       89
101     88
78      85
84      81
22      79
76      78
100     77
59      62
87      57
25      32
66      20
71      11
64       9
19       7
93       5
14       3
102      1
88       1
89       1
Name: cluster, Length: 112, dtype: int64

In [29]:
sentences_clustered.to_csv('output_itr_1.csv')

In [30]:
from sklearn.metrics import accuracy_score
metrics.adjusted_rand_score(sentences_clustered['Risk_tag'],sentences_clustered['cluster'])

KeyError: 'Risk_tag'